In [ ]:
import pandas as pd
import numpy as np
import pathlib
from wmb import *
from ALLCools.plot import *

In [ ]:
mc_annot = cemba.get_mc_annot()
atac_annot = cemba_atac.get_atac_annot()

In [ ]:
def save_index(cells, file_name, downsample=100000, random_state=0):
    cells = pd.Series(cells)
    if cells.size > downsample:
        cells = cells.sample(downsample, random_state=random_state)
    cells.to_csv(file_name, index=None, header=False)

## L1 - Neuron Non-Neuron

In [ ]:
# neuron (not include DG-GC and CB)
mc_neurons = ~mc_annot['L1_annot'].isin(['ODC', 'OPC', 'ASC', 'MGC', 'CB', 'CBX', 'DG'])
atac_neurons = ~atac_annot['L1'].isin([
    'VPIA', 'VLMC', 'MGL', 'PER', 'VEC', 'RGL', 'ASC', 'EPEN', 'BERG', 'OPC',
    'IOL', 'OGC', 'GRC', 'GRANGL'
])

In [ ]:
pathlib.Path('../L1/Neuron').mkdir(exist_ok=True, parents=True)
save_index(mc_annot.sel(cell=mc_neurons).get_index('cell'),
           file_name='../L1/Neuron/mc_cells.txt',
           downsample=10000000,
           random_state=0)

save_index(atac_annot.sel(cell=atac_neurons).get_index('cell'),
           file_name='../L1/Neuron/atac_cells.txt',
           downsample=10000000,
           random_state=0)

In [ ]:
cwd = pathlib.Path().absolute()

with open('Snakefile') as f, open('../L1/Snakefile', 'w') as out_f:
    snakefile_tmp = f.read()
    snakefile_tmp = snakefile_tmp.replace('REPLACE_TEMPLATE_DIR', str(cwd))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_DATASET', 'CEMBA_ATAC')
    snakefile_tmp = snakefile_tmp.replace('REPLACE_CLUSTER_COL', str('L1'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_GROUP_KEY',
                                          str('L2'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_PLOT_KEY',
                                          str('L1_annot'))
    snakefile_tmp = snakefile_tmp.replace(
        'REPLACE_CATEGORICAL_KEY', str(['L1_annot', 'L2', 'DissectionRegion']))
    out_f.write(snakefile_tmp)

## Prepare L2